In [ ]:
!pip install wikipedia

In [ ]:
import random, time, pandas as pd, pyfiglet, wikipedia, warnings
from colorama import Fore, Style, init

# ignoring the parser
warnings.filterwarnings("ignore", category=UserWarning, module="wikipedia")
# initializing colorama
init(autoreset=True, convert=True)

# Load word list from the csv file
easy_df = pd.read_csv("easy_words.csv")
med_df = pd.read_csv("medium_words.csv")
hard_df = pd.read_csv("hard_words.csv")
fruit = pd.read_csv("fruit.csv")
boll_celeb = pd.read_csv("Bolly_celeb.csv")
country = pd.read_csv("Country.csv")
vege = pd.read_csv("Vegatables.csv")

# Converts columns to list basically select the entire row with the first column
easy_word = easy_df.iloc[:,0].dropna().tolist()          ## iloc[:, 0] means select the row
med_word = med_df.iloc[:,0].dropna().tolist()           ## iloc[0] means first row
hard_word = hard_df.iloc[:,0].dropna().tolist()
fruit_word = fruit.iloc[:,0].dropna().tolist()
boll_celeb_word = boll_celeb.iloc[:,0].dropna().tolist()
country_word = country.iloc[:,0].dropna().tolist()
vege_word = vege.iloc[:,0].dropna().tolist()

## Print welcome message!
def welcome_message():
    global name
    name = input(Fore.WHITE + Style.BRIGHT + "Enter your name!: ")
    print(Fore.CYAN + Style.BRIGHT + f"Hey {name}, welcome aboard the Word Guessing Game!")
    time.sleep(0.8)
    print(Fore.BLACK + Style.BRIGHT + "- Guess the secret word using hints.")
    time.sleep(0.6)
    print(Fore.WHITE + Style.BRIGHT +  "- If you nail the right letter in the right spot, it’ll pop up.")
    time.sleep(0.6)
    print(Fore.BLACK + Style.BRIGHT +  "- After 5 wrong shots, you can ask me for a clue.")
    time.sleep(0.6)
    print(Fore.WHITE + Style.BRIGHT +  "- Levels change the word length & how many chances you get.")
    time.sleep(0.6)
    print(Fore.BLACK + Style.BRIGHT +  "- Wanna quit anytime? Type 'quit' or 'q' and poof—you’re out!")
    time.sleep(0.8)
    
# Set attemps for each level
def get_attempts_by_level(level):
    attempts = {'easy': 7, 'medium': 15, 'hard': 20}
    return attempts.get(level, 7)

def get_time_by_word(choose_word):
    if len(choose_word) <= 5:
        return 30
    elif len(choose_word) <= 7:
        return 60
    else:
        return 120

# Taking input for difficulty level
def select_level():
    print(Fore.BLUE + Style.BRIGHT + "Available levels:")
    time.sleep(0.7)
    print(Fore.BLACK + Style.BRIGHT + "1. Easy (3 letters, 7 attempts)")
    time.sleep(0.6)
    print(Fore.WHITE + Style.BRIGHT + "2. Medium (6-7 letters, 15 attempts)")
    time.sleep(0.6)
    print(Fore.BLACK + Style.BRIGHT + "3. Hard (8+ letters, 20 attempts)")
    time.sleep(0.6)
    print(Fore.WHITE + Style.BRIGHT + "4. Or spice it up with: Vegetables  | Fruit  | Country  | Bollywood Celebs ")
    time.sleep(0.8)
    level_map = {
        "easy": easy_word,
        "medium": med_word,
        "hard": hard_word,
    }
    category_map = {
        "fruit": fruit_word,
        "country": country_word,
        "vegetables": vege_word,
        "bollywood": boll_celeb_word,
        "bolly": boll_celeb_word,
        "celebs": boll_celeb_word,
        "celebrities": boll_celeb_word,
        "bollywood celebs": boll_celeb_word,
    }
    while True:
        level = input(Fore.CYAN + "Here’s your menu, star: Easy  | Medium  | Hard  | Catogery: ").lower()
        if level == "hard":
            user = input(Fore.MAGENTA + "Hard mode? Bold choice! You sure? (yes/no): ").lower()
            if user == "yes":
                print(Fore.YELLOW + Style.BRIGHT + "Very well! Don't be annoyed!")
                return random.choice(hard_word).lower(), level, get_attempts_by_level(level), None, None
                time.sleep(0.8)
                
            elif user == "no":
                print(Fore.CYAN + "Cool, no stress! Let’s chill with something lighter ")
                time.sleep(0.8)
                alt_level = input("Easy/Medium or quit: ").lower()

                if alt_level in ("easy", "medium"):
                    return random.choice(level_map[alt_level]).lower(), alt_level, get_attempts_by_level(alt_level), None, None
                elif alt_level in ("quit", "q"):
                    print(Fore.YELLOW + Style.BRIGHT + "Alright, quitting out... thanks for playing!")
                    return None, None, None, None, None
            else:
                print(Fore.RED + Style.BRIGHT + "Invalid Choice! Defauting to Medium level")
                time.sleep(0.8)
                return random.choice(med_word).lower(), 'medium', get_attempts_by_level('medium'), None, None

        elif level in level_map:  # handles easy, medium, fruit, country, etc.
            choose_word = random.choice(level_map[level]).lower()
            max_attempts = get_attempts_by_level(level)
            start_time = None
            max_time = None
            return choose_word, level, max_attempts, None, None
        elif level in category_map:
            choose_word =  random.choice(category_map[level]).lower()
            max_attempts = None
            max_time = get_time_by_word(choose_word)
            start_time = time.time()
            print(Fore.CYAN + f"You have {max_time} seconds to guess this {level} word. Go!") 
            return choose_word, level, None, max_time, start_time
        else:
            print(Fore.RED + "Bruhh! That’s not on the list… rolling with Easy mode by default.")
            time.sleep(0.8)
            choose_word = random.choice(easy_word).lower()
            max_attempts = get_attempts_by_level('easy')
            start_time = None
            max_time = None
            return choose_word, 'easy', max_attempts, None, None
        
#Show hideen letter from the word
def show_hidden_letter(choose_word, wrong_attempts):
    if wrong_attempts == 5:
        user = input(Fore.YELLOW + "Yo, 5 wrong guesses already! Want me to drop a clue? (yes/no): ").lower()
        if user == "yes":
            print("Okay! Here's your clue...")
            time.sleep(1)
            try:
                clue = wikipedia.summary(choose_word, sentences=1)
                print("Clue: ", clue)
                return clue
            except:
                index = random.randint(0, len(choose_word) - 1)
                print(Fore.YELLOW + f"Sneaky peek! Letter at position {index+1} is: {choose_word[index]}")
        elif user == "no":
            print(Fore.CYAN + "Okay okay, no clue then! Let’s keep rollin’ ")
            time.sleep(1)
        else:
            print(Fore.RED + "Heh, invalid choice! Moving on... keep guessing ")
            time.sleep(1)
            
# Main Game Loop
def play_game(choose_word, max_attempts, max_time, start_time):
    attempts = 0
    wrong_attempts = 0
    clue = None
    
    # Show user the number of letters contains in the secret word and max attempts
    display_word = " ".join("_" if c.isalpha() else " " for c in choose_word)
    print(Fore.GREEN + Style.BRIGHT + "\nGame is starting...")
    time.sleep(1)
    print("Word picked! Time to flex your brain ")
    time.sleep(0.6)
    print(Fore.WHITE+ f"The secret word has {len(choose_word)} Here’s how your word is shaping up: {display_word}")
    time.sleep(0.6)
    if max_attempts:
        print(Fore.LIGHTMAGENTA_EX + f"You have {max_attempts} attempts. Good luck!")
        time.sleep(1) 
    elif max_time:
        print(Fore.LIGHTMAGENTA_EX + f"You have {max_time} seconds to guess. Good luck!")
    while True:
        guess = input("Enter your guess: ").lower()
        time.sleep(0.6)
        attempts += 1 

        if max_attempts and attempts > max_attempts:
            print(Fore.RED + pyfiglet.figlet_format("Booooo!! Game Over!"))
            print(Fore.YELLOW + f"Unfortunately! The word you were hunting was: {choose_word}")
            return False

        if max_time and start_time:
            if time.time() - start_time > max_time:
                print(Fore.RED + pyfiglet.figlet_format("Booooo!! Time's Up!"))
                print(Fore.YELLOW + f"The word was: {choose_word}")
                return False
        
        if guess == choose_word:
            print(Fore.GREEN + Style.BRIGHT + f"Niceee! You nailed it in {attempts} tries")
            time.sleep(0.7)
            print(Fore.GREEN + Style.BRIGHT + f"Certainly! the word was {choose_word}.")
            time.sleep(1)
            return True
        elif guess in ("quit", "q"):  # quit game when ever want to end
            print(Fore.YELLOW + "Alrighty, bailing out... cya next time!")
            return None  
        else:
            wrong_attempts += 1

        # Show clue after 5 wrong attempts
        if wrong_attempts == 5:
            show_hidden_letter(choose_word, wrong_attempts)

        # Build hint
        hint = ""
        for i,c in enumerate(choose_word):
            if i < len(guess) and guess[i] == c:
                hint += c + " "
            elif c == " ":
                hint+= "  "
            else:
                hint += ("_" + " ")

        print(Fore.YELLOW + Style.BRIGHT+"Hint: ", hint)
        if max_attempts is not None:
            print(Fore.RED + Style.BRIGHT+f"Attempts left {max_attempts - attempts}")
        else:
            time_left = int(max_time - (time.time() - start_time))
            if time_left > 0:
                print(Fore.RED + Style.BRIGHT + f"Time left: {time_left} seconds")
            else:
                print(Fore.RED + "Time’s up!")
                return False
## Asking user to play again
def playing_again():
    while True:
        ask = input(f"Up for another round, {name}? (yes/no): ").strip().lower()
        if ask not in ("yes", "y"):
            print("Thank you for playing! See you next time!")
            return False
        return True
        
def main():
    welcome_message()
    while True:
        choose_word, difficult_level, max_attempts, max_time, start_time = select_level()
        if not choose_word:
            break
        # get attempts only for the level not category
        max_attempts = get_attempts_by_level(difficult_level) if difficult_level in ('easy', 'medium', 'hard') else None
        result = play_game(choose_word, max_attempts, max_time, start_time)
        if result is None: # Quit fromm mid game 
            break
        if not playing_again():
            break
    print(Fore.YELLOW + "Thanks for vibin’ with me! Catch you next time ")
if __name__ == "__main__":
    main()